In [ ]:
#@title Copyright 2020 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Projects/Satellite_img')

Mounted at /content/drive


In [3]:
import numpy as np
from scipy.stats import norm, gamma, f, chi2
%matplotlib inline
from glob import glob
try: 
  import geopandas
except ModuleNotFoundError:
  !pip install geopandas
  import geopandas as gpd
import folium

     |████████████████████████████████| 1.0 MB 4.0 MB/s 
     |████████████████████████████████| 6.3 MB 38.7 MB/s 
     |████████████████████████████████| 15.4 MB 41.4 MB/s 


Get locations from shp file:

In [4]:
DATA_PATH = 'Markus'
shp_file = glob(os.path.join(DATA_PATH, 'Detections_shp', '*shp'))[0]
avl_polygons = gpd.read_file(shp_file).to_crs(epsg=4326)
location = [avl_polygons.centroid.y.median(), avl_polygons.centroid.x.median()]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  after removing the cwd from sys.path.


In [5]:
_min = avl_polygons.bounds.min()
_min = _min.loc[[i for i in _min.index if 'min' in i]]
_max = avl_polygons.bounds.max()
_max = _max.loc[[i for i in _max.index if 'max' in i]]

In [6]:
coords = [[_min.minx, _min.miny],
          [_max.maxx, _min.miny],
          [_max.maxx, _max.maxy],
          [_min.minx, _max.maxy],
          [_min.minx, _min.miny]]
sub_coords = [[_min.minx, _min.miny], 
             [np.linspace(_min.minx, _max.maxx, 5)[1], _min.miny], 
             [np.linspace(_min.minx, _max.maxx, 5)[1], np.linspace(_min.miny, _max.maxy,5)[1]],
             [_min.minx, np.linspace(_min.miny, _max.maxy,5)[1]],
             [_min.minx, _min.miny]]

In [7]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=qNJcpdKR2TSRgc4oSWJCaW5f2urb8WlpsXzfq150YWM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWio6xXZf34MQgu466U4tyF3QoeVpO0RYa_E_xkObYRD3eomY_cNyjU

Successfully saved authorization token.


Function to calculate the chi square cumulative distribution function and the determinant of a Sentinel-1 image.

In [8]:
def chi2cdf(chi2, df):
    """Calculates Chi square cumulative distribution function for
       df degrees of freedom using the built-in incomplete gamma
       function gammainc().
    """
    return ee.Image(chi2.divide(2)).gammainc(ee.Number(df).divide(2))

def det(im):
    """Calculates determinant of 2x2 diagonal covariance matrix."""
    return im.expression('b(0)*b(1)')

Import of interactive graphics from the _folium_ package:

In [9]:
import folium
from folium.plugins import MousePosition

def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds Earth Engine layers to a folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

## Multitemporal change detection


### A time series

From Central Asia during March and April, 2021

In [10]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          coords
        ]
      }
    }
  ]
}
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": 
        [
          sub_coords
        ]
      }
    }
  ]
}
sub_coords = geoJSON['features'][0]['geometry']['coordinates']
sub_aoi = ee.Geometry.Polygon(sub_coords)

Find relative orbit number and image between March 1, 2021 and April 30, 2021

In [ ]:
from tqdm import tqdm_notebook
orbits = {}
for i in tqdm_notebook(range(1, 181)):
  im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
           .filterBounds(aoi)
           .filterDate(ee.Date('2021-03-01'),ee.Date('2021-04-30'))
           .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
           .filter(ee.Filter.eq('relativeOrbitNumber_start', i))
           .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
           .sort('date'))

  timestamplist = (im_coll.aggregate_array('date')
                 .map(lambda d: ee.String('T').cat(ee.String(d)))
                 .getInfo())
  if len(timestamplist) > 0:
    print(f'relativeOrbitNumber_start: {i}, # of images: {len(timestamplist)}')
    orbits[i] = timestamplist

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/180 [00:00<?, ?it/s]

relativeOrbitNumber_start: 100, # of images: 5
relativeOrbitNumber_start: 173, # of images: 5


In [ ]:
orbits

{100: ['T20210309', 'T20210321', 'T20210402', 'T20210414', 'T20210426'],
 173: ['T20210302', 'T20210314', 'T20210326', 'T20210407', 'T20210419']}

The image collection below covers the months of March, 2021 through April, 2021 at 6-day intervals:

In [11]:
im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
           .filterBounds(aoi)
           .filterDate(ee.Date('2021-03-01'),ee.Date('2021-04-30'))
           .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
           .filter(ee.Filter.eq('relativeOrbitNumber_start', 100))
           .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
           .sort('date'))

timestamplist = (im_coll.aggregate_array('date')
                 .map(lambda d: ee.String('T').cat(ee.String(d)))
                 .getInfo())
timestamplist

['T20210309', 'T20210321', 'T20210402', 'T20210414', 'T20210426']

It will turn out to be convenient to work with a list rather than a collection, so we'll convert the collection to a list and, while we're at it, clip the images to our AOI:

In [12]:
def clip_img(img):
    """Clips a list of images."""
    return ee.Image(img).clip(aoi)

im_list = im_coll.toList(im_coll.size())
im_list = ee.List(im_list.map(clip_img))

im_list.length().getInfo()

5

Here is an RGB composite of the VV bands for three images in early November, after conversion to decibels. Note that some changes, especially those due to flooding, already show up in this representation as colored pixels.

In [13]:

def selectvv(current):
    return ee.Image(current).select('VV')

vv_list = im_list.map(selectvv)
tiles='Stamen Terrain'
location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(location=location, tiles='Stamen Terrain', zoom_start=10)
rgb_images = (ee.Image.rgb(vv_list.get(1), vv_list.get(2), vv_list.get(3))
              .log10().multiply(10))
mp.add_ee_layer(rgb_images, {'min': -20,'max': 0}, 'rgb composite')
mp.add_child(folium.LayerControl())


The off-diagonal elements are mostly small. The not-so-small values can be attributed to sampling error or to the presence of some change pixels in the samples.

#### Dual polarization and an algorithm

With our substitution trick, we can now write down the sequential test for the dual polarization (bivariate) image time series. From Eq. (3.8) we get

$$
Q_k = \prod_{j=2}^k R_j , \quad R_j = \left[{j^{2j}\over (j-1)^{2(j-1)}}{|c_1+\dots +c_{j-1}|^{j-1}|c_j|\over |c_1+\dots +c_j|^j}\right]^m,\quad j = 2\dots k. \tag{3.9}
$$

And of course we have again to use Wilks' Theorem to get the _P_ values, so we work with

$$
-2\log{R_j} = -2m\Big[2(j\log{j}-(j-1)\log(j-1)+(j-1)\log\Big|\sum_{i=1}^{j-1}c_i \Big|+\log|c_j|-j\log\Big|\sum_{i=1}^j c_i\Big|\ \Big] \tag{3.10a}
$$

and

$$
-2\log Q_k = \sum_{j=2}^k -2\log R_j. \tag{3.10b}
$$

The statistic $-2\log R_j$ is approximately chi square distributed with two degrees of freedom. Similarly $-2\log Q_k$ is approximately chi square distributed with $2(k-1)$ degrees of freedom. Readers should satisfy themselves that these numbers are indeed the correct, taking into account that each measurement $c_i$ has two free parameters $|S^a_{vv}|^2$ and $|S^b_{vh}|^2$.

Now for the algorithm:

**The sequential omnibus change detection algorithm**

With a time series of $k$ SAR images $(c_1,c_2,\dots,c_k)$,

1. Set $\ell = k$.
2. Set $s = (c_{k-\ell+1}, \dots c_k)$.
3. Perform the omnibus test $Q_\ell$ for any changes change over $s$.
4. If no significant changes are found, stop.
5. Successively test series $s$ with $R_2, R_3, \dots$ until the first significant change is met for $R_j$.
6. Set $\ell = k-j+1$ and go to 2.

|Table 3.1 |       |       |       |       |       |        |
|----------|-------|-------|-------|-------|-------|--------|
|  $\ell$  | $c_1$ | $c_2$ | $c_3$ | $c_4$ | $c_5$ |        |
| 5        |       | $R^5_2$ | $R^5_3$ | $R^5_4$ | $R^5_5$ | $Q_5$  |
| 4        |       |       | $R^4_2$ | $R^4_3$ | $R^4_4$ | $Q_4$  |
| 3        |       |       |       | $R^3_2$ | $R^3_3$ | $Q_3$  |
| 2        |       |       |       |       | $R^2_2$ | $Q_2$  |


Thus if a change is found, the series is truncated up to the point of change and the testing procedure is repeated for the rest of the series. Take for example a series of $k=5$ images. (See Table 3.1 where, to avoid ambiguity, we add superscript $\ell$ to each $R_j$ test). Suppose there is one change in the second interval only. Then the test sequence is (the asterisk means $H_0$ is rejected)

$$
Q^*_5 \to R^5_2 \to R^{5*}_3 \to Q_3.
$$

If there are changes in the second and last intervals,

$$
Q^*_5 \to R^5_2 \to R^{5*}_3 \to Q^*_3 \to R^3_2 \to R^{3*}_3,
$$

and if there are significant changes in all four intervals,

$$
Q^*_5 \to R^{5*}_2 \to Q^*_4 \to R^{4*}_2 \to Q^*_3 \to R^{3*}_2 \to Q^*_2.
$$

The approach taken in the coding of this algorithm is to pre-calculate  _P_ values for all of the $Q_\ell / R_j$ tests and then, in a second pass, to filter them to determine the points of change.



#### Pre-calculating the _P_ value array

The following code cell performs map operations on the indices $\ell$ and $j$, returning an array of _P_ values for all possible LRT statistics. For example again for $k=5$, the code calculates the _P_ values for each $R_j$ entry in Table 3.1 as a list of lists. Before calculating each row, the time series $c_1, c_2,c_3,c_4, c_5$ is sliced from $k-\ell+1$ to $k$. The last entry in each row is simply the product of the other entries,  $Q_\ell =\prod_{j=2}^\ell R_j.$

The program actually operates on the logarithms of the test statistics, Equations (3.10).


In [15]:
def log_det_sum(im_list, j):
    """Returns log of determinant of the sum of the first j images in im_list."""
    im_ist = ee.List(im_list)
    sumj = ee.ImageCollection(im_list.slice(0, j)).reduce(ee.Reducer.sum())
    return ee.Image(det(sumj)).log()

def log_det(im_list, j):
    """Returns log of the determinant of the jth image in im_list."""
    im = ee.Image(ee.List(im_list).get(j.subtract(1)))
    return ee.Image(det(im)).log()

def pval(im_list, j, m=4.4):
    """Calculates -2logRj for im_list and returns P value and -2logRj."""
    im_list = ee.List(im_list)
    j = ee.Number(j)
    m2logRj = (log_det_sum(im_list, j.subtract(1))
               .multiply(j.subtract(1))
               .add(log_det(im_list, j))
               .add(ee.Number(2).multiply(j).multiply(j.log()))
               .subtract(ee.Number(2).multiply(j.subtract(1))
               .multiply(j.subtract(1).log()))
               .subtract(log_det_sum(im_list,j).multiply(j))
               .multiply(-2).multiply(m))
    pv = ee.Image.constant(1).subtract(chi2cdf(m2logRj, 2))
    return (pv, m2logRj)

def p_values(im_list):
    """Pre-calculates the P-value array for a list of images."""
    im_list = ee.List(im_list)
    k = im_list.length()

    def ells_map(ell):
        """Arranges calculation of pval for combinations of k and j."""
        ell = ee.Number(ell)
        # Slice the series from k-l+1 to k (image indices start from 0).
        im_list_ell = im_list.slice(k.subtract(ell), k)

        def js_map(j):
            """Applies pval calculation for combinations of k and j."""
            j = ee.Number(j)
            pv1, m2logRj1 = pval(im_list_ell, j)
            return ee.Feature(None, {'pv': pv1, 'm2logRj': m2logRj1})

        # Map over j=2,3,...,l.
        js = ee.List.sequence(2, ell)
        pv_m2logRj = ee.FeatureCollection(js.map(js_map))

        # Calculate m2logQl from collection of m2logRj images.
        m2logQl = ee.ImageCollection(pv_m2logRj.aggregate_array('m2logRj')).sum()
        pvQl = ee.Image.constant(1).subtract(chi2cdf(m2logQl, ell.subtract(1).multiply(2)))
        pvs = ee.List(pv_m2logRj.aggregate_array('pv')).add(pvQl)
        return pvs

    # Map over l = k to 2.
    ells = ee.List.sequence(k, 2, -1)
    pv_arr = ells.map(ells_map)

    # Return the P value array ell = k,...,2, j = 2,...,l.
    return pv_arr

#### Filtering the _P_ values

|Table 3.2 |       |       |       |       |       |        |
|----------|-------|-------|-------|-------|-------|--------|
|$i\ $ / $j$|      |     1 |     2 |     3 |     4 |        |
| 1        |       | $P_2$ | $P_3$ | $P_4$ | $P_5$ | $P_{Q5}$  |
| 2        |       |       | $P_2$ | $P_3$ | $P_4$ | $P_{Q4}$  |
| 3        |       |       |       | $P_2$ | $P_3$ | $P_{Q3}$  |
| 4        |       |       |       |       | $P_2$ | $P_{Q2}$  |

The pre-calculated _P_ values in _pv\_arr_ (shown schematically in Table 3.2 for $k=5$) are then scanned in nested iterations over indices $i$ and $j$ to determine the following thematic change maps:

- cmap: the interval of the most recent change, one band, byte values $\in [0,k-1]$,
- smap: the interval of the first change, one band, byte values $\in [0,k-1]$,
- fmap: the number of changes, one band, byte values $\in [0,k-1]$,
- bmap: the changes in each interval, $\ k-1$ bands, byte values $\in [0,1]$).

A boolean variable _median_ is included in the code. Its purpose is to reduce the salt-and-pepper effect in no-change regions, which is at least partly a consequence of the uniform distribution of the _P_ values under $H_0$ (see the section [A note on P values](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-2#a_note_on_p_values) in Part 2). If _median_ is _True_, the _P_ values for each $Q_\ell$ statistic are passed through a $5\times 5$ median filter before being compared with the significance threshold. This is not statistically kosher but probably justifiable if one is only interested in large homogeneous changes, for example flood inundations or deforestation.

Here is the code:

In [16]:
def filter_j(current, prev):
    """Calculates change maps; iterates over j indices of pv_arr."""
    pv = ee.Image(current)
    prev = ee.Dictionary(prev)
    pvQ = ee.Image(prev.get('pvQ'))
    i = ee.Number(prev.get('i'))
    cmap = ee.Image(prev.get('cmap'))
    smap = ee.Image(prev.get('smap'))
    fmap = ee.Image(prev.get('fmap'))
    bmap = ee.Image(prev.get('bmap'))
    alpha = ee.Image(prev.get('alpha'))
    j = ee.Number(prev.get('j'))
    cmapj = cmap.multiply(0).add(i.add(j).subtract(1))
    # Check      Rj?            Ql?                  Row i?
    tst = pv.lt(alpha).And(pvQ.lt(alpha)).And(cmap.eq(i.subtract(1)))
    # Then update cmap...
    cmap = cmap.where(tst, cmapj)
    # ...and fmap...
    fmap = fmap.where(tst, fmap.add(1))
    # ...and smap only if in first row.
    smap = ee.Algorithms.If(i.eq(1), smap.where(tst, cmapj), smap)
    # Create bmap band and add it to bmap image.
    idx = i.add(j).subtract(2)
    tmp = bmap.select(idx)
    bname = bmap.bandNames().get(idx)
    tmp = tmp.where(tst, 1)
    tmp = tmp.rename([bname])
    bmap = bmap.addBands(tmp, [bname], True)
    return ee.Dictionary({'i': i, 'j': j.add(1), 'alpha': alpha, 'pvQ': pvQ,
                          'cmap': cmap, 'smap': smap, 'fmap': fmap, 'bmap':bmap})

def filter_i(current, prev):
    """Arranges calculation of change maps; iterates over row-indices of pv_arr."""
    current = ee.List(current)
    pvs = current.slice(0, -1 )
    pvQ = ee.Image(current.get(-1))
    prev = ee.Dictionary(prev)
    i = ee.Number(prev.get('i'))
    alpha = ee.Image(prev.get('alpha'))
    median = prev.get('median')
    # Filter Ql p value if desired.
    pvQ = ee.Algorithms.If(median, pvQ.focal_median(2.5), pvQ)
    cmap = prev.get('cmap')
    smap = prev.get('smap')
    fmap = prev.get('fmap')
    bmap = prev.get('bmap')
    first = ee.Dictionary({'i': i, 'j': 1, 'alpha': alpha ,'pvQ': pvQ,
                           'cmap': cmap, 'smap': smap, 'fmap': fmap, 'bmap': bmap})
    result = ee.Dictionary(ee.List(pvs).iterate(filter_j, first))
    return ee.Dictionary({'i': i.add(1), 'alpha': alpha, 'median': median,
                          'cmap': result.get('cmap'), 'smap': result.get('smap'),
                          'fmap': result.get('fmap'), 'bmap': result.get('bmap')})

#### Post-processing: The Loewner order

The above change maps are still difficult to interpret. But what about _bmap_, the map of changes detected in each interval? Before we look at them it makes sense to include the direction of change, i.e., the [Loewner order](https://ieeexplore.ieee.org/document/8736751), see [Part 2](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-2#change_direction_the_loewner_order). In the event of significant change at time $j$, we can simply determine the positive or negative definiteness (or indefiniteness) of the difference between consecutive covariance matrix pixels

$$
c_j-c_{j-1},\quad j = 2,\dots,k,
$$

to get the change direction. But we can do better. Instead of subtracting the value for the preceding image, $c_{j-1}$, we can subtract the average over all values up to and including time $j-1$ for which no change has been signalled. For example for $k=5$, suppose there are significant changes in the first and fourth (last) interval. Then to get their directions we examine the differences

$$
c_2-c_1\quad{\rm and}\quad c_5 - (c_2+c_3+c_4)/3.
$$

The running averages can be conveniently determined with the so-called _provisional means algorithm_. The average $\bar c_i$ of the first $i$ images is calculated recursively as

$$
\begin{align*}
\bar c_i &= \bar c_{i-1} + (c_i - \bar c_{i-1})/i \cr
\bar c_1 &= c_1.
\end{align*}
$$

The function _dmap\_iter_ below is iterated over the bands of _bmap_, replacing the values for changed pixels with

- 1 for positive definite differences,
- 2 for negative definite differences,
- 3 for indefinite differences.

In [19]:
def dmap_iter(current, prev):
    """Reclassifies values in directional change maps."""
    prev = ee.Dictionary(prev)
    j = ee.Number(prev.get('j'))
    image = ee.Image(current)
    avimg = ee.Image(prev.get('avimg'))
    diff = image.subtract(avimg)
    # Get positive/negative definiteness.
    posd = ee.Image(diff.select(0).gt(0).And(det(diff).gt(0)))
    negd = ee.Image(diff.select(0).lt(0).And(det(diff).gt(0)))
    bmap = ee.Image(prev.get('bmap'))
    bmapj = bmap.select(j)
    dmap = ee.Image.constant(ee.List.sequence(1, 3))
    bmapj = bmapj.where(bmapj, dmap.select(2))
    bmapj = bmapj.where(bmapj.And(posd), dmap.select(0))
    bmapj = bmapj.where(bmapj.And(negd), dmap.select(1))
    bmap = bmap.addBands(bmapj, overwrite=True)
    # Update avimg with provisional means.
    i = ee.Image(prev.get('i')).add(1)
    avimg = avimg.add(image.subtract(avimg).divide(i))
    # Reset avimg to current image and set i=1 if change occurred.
    avimg = avimg.where(bmapj, image)
    i = i.where(bmapj, 1)
    return ee.Dictionary({'avimg': avimg, 'bmap': bmap, 'j': j.add(1), 'i': i})

We only have to modify the _change\_maps_ function to include the change direction in the _bmap_ image:

In [20]:
def change_maps(im_list, median=False, alpha=0.01):
    """Calculates thematic change maps."""
    k = im_list.length()
    # Pre-calculate the P value array.
    pv_arr = ee.List(p_values(im_list))
    # Filter P values for change maps.
    cmap = ee.Image(im_list.get(0)).select(0).multiply(0)
    bmap = ee.Image.constant(ee.List.repeat(0,k.subtract(1))).add(cmap)
    alpha = ee.Image.constant(alpha)
    first = ee.Dictionary({'i': 1, 'alpha': alpha, 'median': median,
                           'cmap': cmap, 'smap': cmap, 'fmap': cmap, 'bmap': bmap})
    result = ee.Dictionary(pv_arr.iterate(filter_i, first))
    # Post-process bmap for change direction.
    bmap =  ee.Image(result.get('bmap'))
    avimg = ee.Image(im_list.get(0))
    j = ee.Number(0)
    i = ee.Image.constant(1)
    first = ee.Dictionary({'avimg': avimg, 'bmap': bmap, 'j': j, 'i': i})
    dmap = ee.Dictionary(im_list.slice(1).iterate(dmap_iter, first)).get('bmap')
    return ee.Dictionary(result.set('bmap', dmap))

Because of the long delays when the zoom level is changed, it is a lot more convenient to export the change maps to GEE Assets and then examine them, either here in Colab or in the Code Editor. This also means the maps will be shown at the correct scale, irrespective of the zoom level. Here I export all of the change maps as a single image.

In [22]:
temp_data_dir = 'temp'

# Run the algorithm with median filter and at 1% significance.
result = ee.Dictionary(change_maps(im_list, median=True, alpha=0.1))
# Extract the change maps and export to assets.
cmap = ee.Image(result.get('cmap'))
smap = ee.Image(result.get('smap'))
fmap = ee.Image(result.get('fmap'))
bmap = ee.Image(result.get('bmap'))
cmaps = ee.Image.cat(cmap, smap, fmap, bmap).rename(['cmap', 'smap', 'fmap']+timestamplist[1:])

task_config = {
    'fileNamePrefix': 'TimeSeriseCanges_{timestamplist[0]}_to_{timestamplist[-1]}',
    'crs': 'EPSG:4326',
    'scale': 10,
    'maxPixels': 10e10,
    'fileFormat': 'GeoTIFF',
    'skipEmptyTiles': True,
    'region': aoi,
    'folder': temp_data_dir
    }
task = ee.batch.Export.image.toDrive(cmaps, str('image'), **task_config)
task.start()
task.status()


{'creation_timestamp_ms': 1639266022804,
 'description': 'image',
 'id': 'GLRHQXW2Q4R6LIVIL24M6WAW',
 'name': 'projects/earthengine-legacy/operations/GLRHQXW2Q4R6LIVIL24M6WAW',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1639266022804}

The asset  _cmaps_ is shared so we can all access it:

In [23]:
#cmaps = ee.Image('projects/earthengine-community/tutorials/detecting-changes-in-sentinel-1-imagery-pt-3/cmaps')
cmaps = cmaps.updateMask(cmaps.gt(0))

location = aoi.centroid().coordinates().getInfo()[::-1]
palette = ['black', 'red', 'cyan', 'yellow']
mp = folium.Map(location=location, tiles=tiles, zoom_start=11)

mp.add_ee_layer(cmaps.select('T20210321'), {'min': 0,'max': 3, 'palette': palette}, '2021-03-21')
mp.add_ee_layer(cmaps.select('T20210402'), {'min': 0,'max': 3, 'palette': palette}, '2021-04-02')
mp.add_ee_layer(cmaps.select('T20210414'), {'min': 0,'max': 3, 'palette': palette}, '2021-04-14')
mp.add_ee_layer(cmaps.select('T20210426'), {'min': 0,'max': 3, 'palette': palette}, '2021-04-26')

mp.add_child(folium.LayerControl())
fmtr = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"
MousePosition(separator=' | ', prefix="Mouse:",
              lat_formatter=fmtr, lng_formatter=fmtr).add_to(mp)
display(mp)

In [26]:
mp.save('Detection_2021-03-01_to_2021-04-30.html')

Now interpretation is somewhat easier. The negative definite (cyan) changes correspond to decreases in intensity of _VV_ and _VH_ reflectance and likely due ti increase in sniwdepth. The positive definite changes (red), which overlay the areas where there is an avalanche debrias.

### Outlook

Without reliable ground truth, we can't claim that color changes in the map will be helpful for avalanche detection.  